In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os

In [2]:
amy = pd.read_excel('./data_fix/Amyloid_SUVR.xlsx')
dt = pd.read_excel('./data_fix/DT_thickness.xlsx')
fdg = pd.read_excel('./data_fix/FDG_SUVR.xlsx')
tau = pd.read_excel('./data_fix/Tau_SUVR.xlsx')

In [3]:
dt.tail()

,Subject,PTID,DX,Node 1,Node 2,Node 3,Node 4,Node 5,Node 6,Node 7,...,Node 151,Node 152,Node 153,Node 154,Node 155,Node 156,Node 157,Node 158,Node 159,Node 160
2321,I829304,002_S_1280,CN,1.988350,2.354034,2.300563,2.496365,2.425688,2.640970,2.406805,...,0.003291,0.007645,0.005240,0.001309,0.001586,0.003825,0.003327,0.005999,0.005169,0.001449
2322,I989328,002_S_1261,CN,2.468522,2.355803,2.464366,2.512443,2.521713,2.605293,2.595708,...,0.002416,0.005817,0.003987,0.001399,0.001006,0.003559,0.002523,0.005756,0.004041,0.001403
2323,I843517,002_S_1155,MCI,2.254826,2.095187,2.065084,2.533945,2.483784,2.463816,2.240186,...,0.003492,0.006310,0.005127,0.001218,0.001119,0.003591,0.003180,0.005687,0.004938,0.001220
2324,I995512,002_S_1155,MCI,2.277251,2.235792,2.233274,2.576189,2.408753,2.516638,2.282218,...,0.003285,0.006396,0.004902,0.001225,0.001455,0.003760,0.003615,0.005894,0.004923,0.001176
2325,I863064,002_S_0413,CN,2.244213,2.682198,2.360108,2.587087,2.555972,2.562441,2.687349,...,0.003203,0.005925,0.003798,0.001045,0.001651,0.003327,0.003284,0.005395,0.003735,0.001121


In [4]:
dt.insert(2, 'label', 0)

In [105]:
dt.loc[5, 'label']

0

In [5]:
for _ in range(len(dt)):
    if dt.loc[_, 'DX'] == 'CN' or dt.loc[_, 'DX'] == 'SMC':
        dt.loc[_, 'label'] = 0
    elif dt.loc[_, 'DX'] == 'MCI' or dt.loc[_, 'DX'] == 'LMCI' or dt.loc[_, 'DX'] == 'EMCI':
        dt.loc[_, 'label'] = 1
    elif dt.loc[_, 'DX'] == 'AD':
        dt.loc[_, 'label'] = 2

In [14]:
dt_148 = dt.iloc[:, :-12]

In [15]:
dt_148.to_csv('./data_fix/DT_thickness_labeling_148.csv', index = False)

In [53]:
diagnosis = dt[['Subject', 'PTID', 'DX']]

In [17]:
all_adjs = os.listdir('./matrices2326/')

In [41]:
all_adjs[0][:10]

'009_S_4388'

In [45]:
all_adjs[0][11:]

'I880136'

In [117]:
dt[all_adjs[0][:10] == dt['PTID']].iloc[:, 3:].values.shape

(1, 161)

In [123]:
dt.to_csv('./data_fix/DT_thickness_labeling.csv', index = False)

In [18]:
adj = np.loadtxt(f'./matrices2326/{all_adjs[0]}', dtype = int)

In [22]:
adj[:148, :148].shape

(148, 148)

In [119]:
node_list = []
adj_list = []
label_list = []

for _ in range(len(all_adjs)):
    node = dt[all_adjs[_][11:] == dt['Subject']].iloc[:, 4:].to_numpy()
    label = dt[all_adjs[_][11:] == dt['Subject']].iloc[:, 2].to_numpy()
    adj = np.loadtxt(f'./matrices2326/{all_adjs[_]}', dtype = int)
    
    node_list.append(node)
    label_list.append(label)
    adj_list.append(adj)


In [130]:
node_np = np.asarray(node_list)
label_np = np.asarray(label_list)
adj_np = np.asarray(adj_list)

In [135]:
import torch
node_tensor = torch.tensor(node_np.transpose((0,2,1)), dtype=torch.float32)

In [138]:
adj = np.loadtxt('./matrices2326/002_S_0413_I863064')

In [2]:
import torch

In [3]:
def smart_sort(x, permutation):
    d1, d2 = x.size()
    ret = x[
        torch.arange(d1).unsqueeze(1).repeat((1, d2)).flatten(),
        permutation.flatten()
    ].view(d1, d2)
    return ret

In [10]:
a = torch.tensor([[1, 0, 2, 2, 1], [0, 2, 1, 2, 0]])
b = torch.tensor([[3, 2, 4, 1, 0], [2, 1, 4, 3, 3]])

In [17]:
d1, d2 = 160, 160

In [18]:
torch.arange(d1).unsqueeze(1).repeat((1, d2)).flatten()

tensor([  0,   0,   0,  ..., 159, 159, 159])

In [15]:
b.flatten()

tensor([3, 2, 4, 1, 0, 2, 1, 4, 3, 3])

In [14]:
a[torch.arange(d1).unsqueeze(1).repeat((1, d2)).flatten(), b.flatten()].view(d1, d2)

tensor([[2, 2, 1, 0, 1],
        [1, 2, 0, 2, 2]])

In [31]:
t = torch.linspace(0,50,1000)
logsnr_min = -5
logsnr_max = 5

In [32]:
limit_max = torch.atan(torch.exp(-0.5 * torch.tensor(logsnr_max)))

In [33]:
limit_max

tensor(0.0819)

In [34]:
limit_min = torch.atan(torch.exp(-0.5 * torch.tensor(logsnr_min))) - limit_max

In [35]:
limit_min

tensor(1.4070)

In [48]:
limit_min * t + limit_max

tensor([0.2195, 0.2207, 0.2218, 0.2229, 0.2241, 0.2252, 0.2263, 0.2275, 0.2286,
        0.2297, 0.2309, 0.2320, 0.2331, 0.2343, 0.2354, 0.2365, 0.2377, 0.2388,
        0.2399, 0.2411, 0.2422, 0.2433, 0.2445, 0.2456, 0.2467, 0.2479, 0.2490,
        0.2501, 0.2513, 0.2524, 0.2535, 0.2547, 0.2558, 0.2569, 0.2581, 0.2592,
        0.2603, 0.2614, 0.2626, 0.2637, 0.2648, 0.2660, 0.2671, 0.2682, 0.2694,
        0.2705, 0.2716, 0.2728, 0.2739, 0.2750, 0.2762, 0.2773, 0.2784, 0.2796,
        0.2807, 0.2818, 0.2830, 0.2841, 0.2852, 0.2864, 0.2875, 0.2886, 0.2898,
        0.2909, 0.2920, 0.2932, 0.2943, 0.2954, 0.2966, 0.2977, 0.2988, 0.3000,
        0.3011, 0.3022, 0.3034, 0.3045, 0.3056, 0.3068, 0.3079, 0.3090, 0.3102,
        0.3113, 0.3124, 0.3136, 0.3147, 0.3158, 0.3170, 0.3181, 0.3192, 0.3204,
        0.3215, 0.3226, 0.3238, 0.3249, 0.3260, 0.3272, 0.3283, 0.3294, 0.3306,
        0.3317, 0.3328, 0.3340, 0.3351, 0.3362, 0.3374, 0.3385, 0.3396, 0.3408,
        0.3419, 0.3430, 0.3441, 0.3453, 

In [40]:
logsnr = torch.tan((limit_min * t / 50 + limit_max)) * 2
logsnr_flip = torch.tan((limit_min * (1-t) + limit_max)) * 2

In [39]:
logsnr = -2 * torch.log(torch.tan(limit_min * t + limit_max))
logsnr_flip = -2 * torch.log(torch.tan(limit_min * (1 - t) + limit_max))

In [41]:
logsnr

tensor([ 0.1642,  0.1670,  0.1698,  0.1727,  0.1755,  0.1784,  0.1812,  0.1840,
         0.1869,  0.1897,  0.1926,  0.1954,  0.1982,  0.2011,  0.2039,  0.2068,
         0.2096,  0.2125,  0.2153,  0.2182,  0.2210,  0.2239,  0.2267,  0.2296,
         0.2324,  0.2353,  0.2382,  0.2410,  0.2439,  0.2467,  0.2496,  0.2525,
         0.2553,  0.2582,  0.2610,  0.2639,  0.2668,  0.2696,  0.2725,  0.2754,
         0.2782,  0.2811,  0.2840,  0.2869,  0.2897,  0.2926,  0.2955,  0.2984,
         0.3013,  0.3041,  0.3070,  0.3099,  0.3128,  0.3157,  0.3186,  0.3215,
         0.3243,  0.3272,  0.3301,  0.3330,  0.3359,  0.3388,  0.3417,  0.3446,
         0.3475,  0.3504,  0.3533,  0.3562,  0.3591,  0.3620,  0.3650,  0.3679,
         0.3708,  0.3737,  0.3766,  0.3795,  0.3824,  0.3854,  0.3883,  0.3912,
         0.3941,  0.3971,  0.4000,  0.4029,  0.4059,  0.4088,  0.4117,  0.4147,
         0.4176,  0.4205,  0.4235,  0.4264,  0.4294,  0.4323,  0.4353,  0.4382,
         0.4412,  0.4441,  0.4471,  0.45

In [61]:
alpha = torch.sqrt(torch.sigmoid(logsnr))
sigma = torch.sqrt(torch.sigmoid(-logsnr))

# alpha = logsnr_flip
# sigma = logsnr

In [53]:
torch.sigmoid(logsnr)

tensor([0.9526, 0.9521, 0.9516, 0.9511, 0.9506, 0.9501, 0.9496, 0.9491, 0.9486,
        0.9481, 0.9476, 0.9471, 0.9466, 0.9461, 0.9456, 0.9451, 0.9446, 0.9441,
        0.9435, 0.9430, 0.9425, 0.9420, 0.9414, 0.9409, 0.9404, 0.9398, 0.9393,
        0.9387, 0.9382, 0.9376, 0.9371, 0.9365, 0.9360, 0.9354, 0.9349, 0.9343,
        0.9338, 0.9332, 0.9326, 0.9321, 0.9315, 0.9309, 0.9303, 0.9298, 0.9292,
        0.9286, 0.9280, 0.9274, 0.9268, 0.9262, 0.9256, 0.9251, 0.9245, 0.9239,
        0.9233, 0.9226, 0.9220, 0.9214, 0.9208, 0.9202, 0.9196, 0.9190, 0.9184,
        0.9177, 0.9171, 0.9165, 0.9159, 0.9152, 0.9146, 0.9140, 0.9133, 0.9127,
        0.9120, 0.9114, 0.9108, 0.9101, 0.9095, 0.9088, 0.9082, 0.9075, 0.9068,
        0.9062, 0.9055, 0.9049, 0.9042, 0.9035, 0.9029, 0.9022, 0.9015, 0.9008,
        0.9002, 0.8995, 0.8988, 0.8981, 0.8974, 0.8967, 0.8960, 0.8954, 0.8947,
        0.8940, 0.8933, 0.8926, 0.8919, 0.8912, 0.8904, 0.8897, 0.8890, 0.8883,
        0.8876, 0.8869, 0.8862, 0.8854, 

In [62]:
sigma

tensor([0.0818, 0.0832, 0.0846, 0.0860, 0.0874, 0.0888, 0.0902, 0.0916, 0.0930,
        0.0944, 0.0958, 0.0972, 0.0986, 0.1000, 0.1014, 0.1028, 0.1042, 0.1056,
        0.1070, 0.1084, 0.1098, 0.1112, 0.1126, 0.1140, 0.1154, 0.1168, 0.1182,
        0.1196, 0.1210, 0.1224, 0.1238, 0.1252, 0.1266, 0.1280, 0.1294, 0.1308,
        0.1322, 0.1336, 0.1350, 0.1364, 0.1378, 0.1392, 0.1406, 0.1420, 0.1434,
        0.1448, 0.1462, 0.1476, 0.1489, 0.1503, 0.1517, 0.1531, 0.1545, 0.1559,
        0.1573, 0.1587, 0.1601, 0.1615, 0.1629, 0.1642, 0.1656, 0.1670, 0.1684,
        0.1698, 0.1712, 0.1726, 0.1740, 0.1754, 0.1767, 0.1781, 0.1795, 0.1809,
        0.1823, 0.1837, 0.1851, 0.1864, 0.1878, 0.1892, 0.1906, 0.1920, 0.1933,
        0.1947, 0.1961, 0.1975, 0.1989, 0.2003, 0.2016, 0.2030, 0.2044, 0.2058,
        0.2071, 0.2085, 0.2099, 0.2113, 0.2127, 0.2140, 0.2154, 0.2168, 0.2182,
        0.2195, 0.2209, 0.2223, 0.2237, 0.2250, 0.2264, 0.2278, 0.2291, 0.2305,
        0.2319, 0.2332, 0.2346, 0.2360, 

In [63]:
alpha

tensor([0.9966, 0.9965, 0.9964, 0.9963, 0.9962, 0.9960, 0.9959, 0.9958, 0.9957,
        0.9955, 0.9954, 0.9953, 0.9951, 0.9950, 0.9948, 0.9947, 0.9946, 0.9944,
        0.9943, 0.9941, 0.9939, 0.9938, 0.9936, 0.9935, 0.9933, 0.9932, 0.9930,
        0.9928, 0.9926, 0.9925, 0.9923, 0.9921, 0.9920, 0.9918, 0.9916, 0.9914,
        0.9912, 0.9910, 0.9908, 0.9907, 0.9905, 0.9903, 0.9901, 0.9899, 0.9897,
        0.9895, 0.9893, 0.9891, 0.9888, 0.9886, 0.9884, 0.9882, 0.9880, 0.9878,
        0.9876, 0.9873, 0.9871, 0.9869, 0.9866, 0.9864, 0.9862, 0.9860, 0.9857,
        0.9855, 0.9852, 0.9850, 0.9848, 0.9845, 0.9843, 0.9840, 0.9838, 0.9835,
        0.9832, 0.9830, 0.9827, 0.9825, 0.9822, 0.9819, 0.9817, 0.9814, 0.9811,
        0.9809, 0.9806, 0.9803, 0.9800, 0.9797, 0.9795, 0.9792, 0.9789, 0.9786,
        0.9783, 0.9780, 0.9777, 0.9774, 0.9771, 0.9768, 0.9765, 0.9762, 0.9759,
        0.9756, 0.9753, 0.9750, 0.9747, 0.9744, 0.9740, 0.9737, 0.9734, 0.9731,
        0.9727, 0.9724, 0.9721, 0.9718, 

In [604]:
alpha_s = alpha[0]
alpha_t = alpha[1]
sigma_s = sigma[0]
sigma_t = sigma[1]

In [623]:
alpha_ts = alpha_t / alpha_s
alpha_st = 1 / alpha_ts
sigma_ts2 = (sigma_t ** 2) - ((alpha_ts ** 2) * (sigma_s ** 2))

tmp1 = sigma_s ** 2
tmp2 = (sigma_t ** 2) / (alpha_ts ** 2) - (sigma_s ** 2)
sigma2_denoise = 1 / ((1 / tmp1) + (1 / tmp2))

In [624]:
coeff1 = sigma2_denoise * alpha_ts / sigma_ts2
coeff2 = sigma2_denoise * alpha_st / (sigma_s ** 2)

In [607]:
tmp1

tensor(0.1991)

In [608]:
tmp2

tensor(0.0043)

In [609]:
sigma_ts2

tensor(0.0042)

In [610]:
sigma2_denoise

tensor(0.0042)

In [611]:
coeff1

tensor(0.9842)

In [612]:
coeff2

tensor(0.0212)

---------------------------

In [613]:
alpha_s = alpha[-500]
alpha_t = alpha[-499]
sigma_s = sigma[-500]
sigma_t = sigma[-499]

In [616]:
tmp1

tensor(4.0091)

In [617]:
tmp2

tensor(0.0365)

In [618]:
sigma_ts2

tensor(0.0363)

In [619]:
sigma2_denoise

tensor(0.0362)

In [620]:
coeff1

tensor(0.9932)

In [621]:
coeff2

tensor(0.0090)

---------------

In [622]:
alpha_s = alpha[-2]
alpha_t = alpha[-1]
sigma_s = sigma[-2]
sigma_t = sigma[-1]

In [625]:
tmp1

tensor(79.4923)

In [626]:
tmp2

tensor(1.7088)

In [627]:
sigma_ts2

tensor(1.6907)

In [628]:
sigma2_denoise

tensor(1.6728)

In [629]:
coeff1

tensor(0.9842)

In [630]:
coeff2

tensor(0.0212)

-----------------

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
from torch.utils.data import TensorDataset, DataLoader 
import torchvision
import torchvision.transforms as T
import glob
import torch.nn.functional as F

In [900]:
data_transforms = [T.Resize((64, 64)), T.RandomHorizontalFlip(), T.ToTensor(), T.Lambda(lambda t: (t * 2) - 1)]
data_transform = T.Compose(data_transforms)

image_folder = torchvision.datasets.ImageFolder(root = '/root/proj/dataset/img_with_gene', transform = data_transform)

train_size = int(0.8 * len(image_folder))
test_size = len(image_folder) - train_size

# train_dataset, test_dataset = torch.utils.data.random_split(image_folder, [train_size, test_size])

filenames = glob.glob('/root/proj/dataset/img_with_gene' + '/*/*')

# idx = image_folder.indices
# test_idx = test_dataset.indices
id_list = []
# test_id = []
for i in range(len(image_folder)):
    id_list.append(filenames[i][-14:-4])
# for j in test_idx:
#     test_id.append(filenames[j][-14:-4])

snp = pd.read_csv('/root/proj/dataset/adni1_go2_qc_dropna_with_apoe_adnimerge_mri.csv')
snp_list = []
# test_snp = []

for i in range(len(id_list)):
    temp = snp[snp['PTID'] == id[i]]
    temp = temp.iloc[:, 1:-2].to_numpy()
    snp_list.append(temp.reshape(temp.shape[1]))

snp_np = np.array(snp_list)
snp_tensor = torch.tensor(snp_np, dtype = torch.long)
snp_tensor = F.one_hot(snp_tensor, num_classes = 3).to(torch.float32)

In [884]:
len(image_folder)

691

In [877]:
loader = DataLoader(image_folder, batch_size = 128, shuffle = False)

In [910]:
snp_tensor.shape

torch.Size([691, 115, 3])

In [917]:
img_list = []
label_list = []
for i, j in enumerate(image_folder):
    img = j[0]
    label = torch.tensor(j[1])
    img_list.append(img)
    label_list.append(label)

In [921]:
img_tensor = torch.stack(img_list)
label_tensor = torch.stack(label_list)

In [923]:
img_tensor.shape, label_tensor.shape, snp_tensor.shape

(torch.Size([691, 3, 64, 64]), torch.Size([691]), torch.Size([691, 115, 3]))

In [924]:
ds = TensorDataset(img_tensor, snp_tensor, label_tensor)

In [3]:
t_min = torch.tensor([0])
t_max = torch.tensor([20])

In [4]:
t = torch.exp(torch.linspace(torch.log(t_min + 0.5).item(), torch.log(t_max).item(),100))

In [39]:
t.shape

torch.Size([1000])

In [9]:
t = torch.cat([t_min, t])

In [6]:
image_size = 28
freqs = torch.pi*torch.linspace(0, image_size-1, image_size) / image_size
frequencies_squared = freqs[:, None]**2 + freqs[None, :]**2

In [14]:
steps = torch.randint(1, 100, (10,))

In [23]:
t = t[:, None, None]

In [30]:
frequencies_squared

tensor([[0.0000e+00, 1.2589e-02, 5.0355e-02, 1.1330e-01, 2.0142e-01, 3.1472e-01,
         4.5320e-01, 6.1685e-01, 8.0568e-01, 1.0197e+00, 1.2589e+00, 1.5232e+00,
         1.8128e+00, 2.1275e+00, 2.4674e+00, 2.8325e+00, 3.2227e+00, 3.6382e+00,
         4.0788e+00, 4.5446e+00, 5.0355e+00, 5.5517e+00, 6.0930e+00, 6.6595e+00,
         7.2511e+00, 7.8680e+00, 8.5100e+00, 9.1772e+00],
        [1.2589e-02, 2.5178e-02, 6.2944e-02, 1.2589e-01, 2.1401e-01, 3.2731e-01,
         4.6578e-01, 6.2944e-01, 8.1827e-01, 1.0323e+00, 1.2715e+00, 1.5358e+00,
         1.8254e+00, 2.1401e+00, 2.4800e+00, 2.8451e+00, 3.2353e+00, 3.6507e+00,
         4.0914e+00, 4.5571e+00, 5.0481e+00, 5.5642e+00, 6.1056e+00, 6.6721e+00,
         7.2637e+00, 7.8806e+00, 8.5226e+00, 9.1898e+00],
        [5.0355e-02, 6.2944e-02, 1.0071e-01, 1.6365e-01, 2.5178e-01, 3.6507e-01,
         5.0355e-01, 6.6721e-01, 8.5604e-01, 1.0700e+00, 1.3092e+00, 1.5736e+00,
         1.8631e+00, 2.1779e+00, 2.5178e+00, 2.8828e+00, 3.2731e+00, 3.688

In [29]:
torch.exp(- frequencies_squared * t)[-1]

tensor([[1.0000e+00, 7.7742e-01, 3.6528e-01, 1.0373e-01, 1.7803e-02, 1.8466e-03,
         1.1577e-04, 4.3864e-06, 1.0045e-07, 1.3902e-09, 1.1628e-11, 5.8787e-14,
         1.7962e-16, 3.3169e-19, 3.7019e-22, 2.4971e-25, 1.0180e-28, 2.5082e-32,
         3.7350e-36, 3.3615e-40, 1.8217e-44, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [7.7742e-01, 6.0438e-01, 2.8397e-01, 8.0640e-02, 1.3840e-02, 1.4356e-03,
         9.0000e-05, 3.4101e-06, 7.8089e-08, 1.0808e-09, 9.0402e-12, 4.5702e-14,
         1.3964e-16, 2.5786e-19, 2.8779e-22, 1.9413e-25, 7.9140e-29, 1.9499e-32,
         2.9037e-36, 2.6133e-40, 1.4013e-44, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [3.6528e-01, 2.8397e-01, 1.3343e-01, 3.7889e-02, 6.5029e-03, 6.7453e-04,
         4.2287e-05, 1.6022e-06, 3.6691e-08, 5.0780e-10, 4.2476e-12, 2.1473e-14,
         6.5611e-17, 1.2116e-19, 1.3522e-22, 9.1212e-26, 3.7184e-29, 9.161

In [44]:
import torch_dct as dct
x = torch.randn(10,)

In [45]:
dct.dct(x)

tensor([ 3.3675,  3.5616, -5.1065,  6.3231,  1.7374,  1.0003, -1.3340,  0.5421,
        12.6905, -1.3913])

In [58]:
torch.randint(0, 10, (5,))

tensor([1, 8, 9, 0, 8])

In [6]:
scales = torch.nn.Parameter(torch.FloatTensor(3).uniform_(0.0, 20.0), requires_grad=False)

In [22]:
scales

Parameter containing:
tensor([15.4867,  1.1096,  7.2503])

In [18]:
L = torch.tensor([[1, 2], [3, 4], [5, 6]])

In [19]:
L

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [21]:
torch.einsum('bn, s -> bsn', L, scales[1:])

tensor([[[ 1.1096,  2.2191],
         [ 7.2503, 14.5006]],

        [[ 3.3287,  4.4383],
         [21.7510, 29.0013]],

        [[ 5.5478,  6.6574],
         [36.2516, 43.5019]]])